<a href="https://colab.research.google.com/github/asigalov61/Amazing-GPT2-Piano/blob/master/Amazing_GPT2_Piano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazing GPT2 Piano (w/MTM 3.5)

***

## GPT2-based Symbolic Music Artificial Intelligence Model Creator/Trainer

### Multi-Track, Multi-Instrumental, MIDI-TXT-MIDI

***

1) Credit for char-based GPT2 implementation used in this colab goes out to Andrej Karpathy: https://github.com/karpathy/minGPT

2) Credit for very nice Arc diagram MIDI visualizer goes out to J. Brent Runyan: https://github.com/j-brent/arc-diagrams

***

#### Project Los Angeles

#### Tegridy Code 2020

***

# Setup Environment, clone needed repos, and install all required dependencies

In [ ]:
#@title Clone minGPT repo and install all dependencies (run only once per session)
!git clone https://github.com/asigalov61/minGPT
!git clone https://github.com/asigalov61/arc-diagrams
!pip install pyknon
!pip install pretty_midi
!pip install pypianoroll
!pip install mir_eval
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 /content/font.sf2

!curl -L "https://github.com/asigalov61/MIDI-TXT-MIDI/raw/master/MIDI.py" > 'MIDI.py'

!mkdir Dataset

In [ ]:
#@title Import all modules and setup logging
%cd /content/minGPT
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

import numpy as np
import torchvision
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset

import keras
from keras.utils import to_categorical

import time
import math

from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig
from mingpt.utils import sample

import tqdm.auto

# For plotting
import mido
import librosa
import pretty_midi
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
import mir_eval.display
import librosa.display
%matplotlib inline

from mido import MidiFile


from midi2audio import FluidSynth

from google.colab import output, drive

from IPython.display import display, Javascript, HTML, Audio, Image

# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%cd /content/arc-diagrams/
from arc_diagram import plot_arc_diagram

dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print('Available Processing Device is:', device)
%cd /content/

# Upload/download and process MIDI dataset

In [ ]:
#@title (OPTION 1) Download Tegridy special Piano/Violin MIDI dataset
%cd /content/Dataset/
!rm *.mid 
!rm *.midi
!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!unzip '/content/Dataset/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!rm '/content/Dataset/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
%cd /content/

In [ ]:
#@title (OPTION 2) Tiny Karaoke Precision MIDI subset (CC-BY-NC-SA)
%cd /content/Dataset/
!rm *.mid 
!rm *.midi
!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tiny-Karaoke-Precision-MIDI-Subset-CC-BY-NC-SA.zip'
!unzip '/content/Dataset/Tiny-Karaoke-Precision-MIDI-Subset-CC-BY-NC-SA.zip'
!rm '/content/Dataset/Tiny-Karaoke-Precision-MIDI-Subset-CC-BY-NC-SA.zip'
%cd /content/

In [ ]:
#@title Process MIDI to TXT (MIDI-TXT-MIDI v.3.5)
encoding_type = "score-karaoke-one-byte-encoding" #@param ["score-karaoke-one-byte-encoding", "score-special-one-byte-encoding"]
enable_sampling = False #@param {type:"boolean"}
sample_length_in_MIDI_events = 1681 #@param {type:"slider", min:0, max:10000, step:1}
advanced_events = True #@param {type:"boolean"}
allow_tempo_changes = True #@param {type:"boolean"}
allow_control_change = True #@param {type:"boolean"}
karaoke = True #@param {type:"boolean"}
debug = False #@param {type:"boolean"}
score2mil = True #@param {type:"boolean"}

%cd /content/

# MIDI Dataset to txt dataset converter 
import MIDI
import os
import numpy as np
import tqdm.auto

if os.path.exists("Dataset.txt"):
  os.remove("Dataset.txt")
  print('Removing old Dataset...')
else:
  print("Creating new Dataset file...")



def write_notes(file_address):
      u = 0
      score = []
      melody = []
      result = []
      midi_file = open(file_address, 'rb')
      if debug: print('Processing File:', file_address)

      if encoding_type == 'score-karaoke-one-byte-encoding':
        score = MIDI.midi2score(midi_file.read())
        midi_file.close()
        score1 = MIDI.grep(score, [0, 2, 4, 6, 8, 10, 12, 14])
        score2 = MIDI.grep(score, [1, 3, 5, 7, 9, 11, 13, 15])
        #score1 = MIDI.grep(score, [0, 2])
        #score2 = MIDI.grep(score, [1, 3])
        #print(score)

        def twolists(list1, list2):
            newlist = []
            a1 = len(list1)
            a2 = len(list2)

            for i in range(max(a1, a2)):
                if i < a1:
                    newlist.append(list1[i])
                if i < a2:
                    newlist.append(list2[i])

            return newlist

        
        #melody = [score[0], score[1]]
        melody = [score1[0], twolists(score1[1], score2[1])]
        #print(melody)

        itrack = 1
        file = open('Dataset.txt', 'a')
        file.write('H d0 tMIDI-TXT-MIDI-Textual-Music-Dataset ')
        while itrack < len(melody):
            for event in melody[itrack]:
                if event[0] == 'note':
                  file.write('N' + ' d' + str(event[1]) + ' D' + str(event[2]) + ' C' + str(event[3]) + ' n' + str(event[4]) + ' V' + str(event[5]) + ' ')
                if event[0] == 'lyric' or event[0] == 'text_event':
                  file.write('L' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            itrack += 1
          

        file.close()
        if debug:
          print('File:', midi_file, 'Number of skipped events: ', u)


      if encoding_type == 'score-special-one-byte-encoding':
        score = MIDI.midi2score(midi_file.read())
        if debug: print(score)
        midi_file.close()
        score1 = MIDI.grep(score, [0, 2, 4, 6, 8, 10, 12, 14])
        score2 = MIDI.grep(score, [1, 3, 5, 7, 9, 11, 13, 15])
        #print(score)

        result = []

        def twolists(list1, list2):
            newlist = []
            a1 = len(list1)
            a2 = len(list2)

            for i in range(max(a1, a2)):
                if i < a1:
                    newlist.append(list1[i])
                if i < a2:
                    newlist.append(list2[i])

            return newlist

        result = [score1[0], twolists(score1[1], score2[1])]
        score = result
        # ['note', start_time, duration, channel, note, velocity]

        itrack = 1
        


        notes = []

        tokens = []

        this_channel_has_note = False

        file = open('Dataset.txt', 'a')
        file.write('H d0 tMIDI-TXT-MIDI-Textual-Music-Dataset ')
        while itrack < len(score):
            for event in score[itrack]:

                if event[0] == 'note':
                    this_channel_has_note = True
                    notes.append(event[4])
                    
                    tokens.append([event[5], event[3], event[2], event[1]])
                    file.write('N' + ' d' + str(event[1]) + ' D' + str(event[2]) + ' C' + str(event[3]) + ' n' + str(event[4]) + ' V' + str(event[5]) + ' ')
                
            itrack += 1
            if not this_channel_has_note:
              u+=1
              if debug: 
                print('Uknown Event: ', event[0])

            if this_channel_has_note and len(notes) > sample_length_in_MIDI_events:
              if enable_sampling:
                break
          

        file.close()
        if debug:
          print('File:', midi_file, 'Number of skipped events: ', u)


dataset_addr = "Dataset"
files = os.listdir(dataset_addr)
for file in tqdm.auto.tqdm(files):
    path = os.path.join(dataset_addr, file)
    write_notes(path)
#print('Done!')
#print('Number of skipped events: ', u)

# Setup and Intialize the Model

In [ ]:
#@title Setup functions and procedures
model_attention_span_in_tokens = 256 #@param {type:"slider", min:0, max:512, step:16}

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

        
block_size = model_attention_span_in_tokens # spatial extent of the model for its context

In [ ]:
#@title Specify input text file with training data (do not worry, any text format is fine)
full_path_to_training_text_file = "/content/Dataset.txt" #@param {type:"string"}
text = open(full_path_to_training_text_file, 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

In [ ]:
#@title Create GPT2 model
model_embed_size = 256 #@param {type:"slider", min:0, max:1024, step:64}
number_of_heads = 16 #@param {type:"slider", min:1, max:16, step:1}
number_of_layers = 4 #@param {type:"slider", min:1, max:16, step:1}


mconf = GPTConfig(train_dataset.vocab_size, 
                  train_dataset.block_size,
                  n_layer=number_of_layers, 
                  n_head=number_of_heads, 
                  n_embd=model_embed_size)

model = GPT(mconf)

In [ ]:
#@title Setup all training parameters
number_of_training_epochs = 5 #@param {type:"slider", min:1, max:100, step:1}
training_batch_size = 160 #@param {type:"slider", min:0, max:160, step:4}
model_learning_rate = 6e-4 #@param {type:"number"}
# initialize a trainer instance and kick off training

tconf = TrainerConfig(max_epochs=number_of_training_epochs, 
                      batch_size=training_batch_size, 
                      learning_rate=model_learning_rate,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)

# Train the Model or load the existing pre-trained model checkpoint

In [ ]:
#@title (OPTION 1) Train the model
%cd /content/
trainer.train()

In [ ]:
#@title Plot Positional Embeddings

# visualize some of the learned positional embeddings, maybe they contain structure
plt.figure(figsize=(18, 1))  
ci = model.pos_emb.data[0, :, 0].cpu()
zci = torch.cat((torch.tensor([0.0]), ci)) # pre-cat a zero
plt.imshow(zci.view(1, block_size+1).numpy())
plt.axis('off')

In [ ]:
#@title Save/Re-Save the model from memory
%cd /content/
torch.save(model, 'trained-model.pth')

In [ ]:
#@title (OPTION 2) Load existing model/checkpoint
full_path_to_model_checkpoint = "/content/trained-model.pth" #@param {type:"string"}
model = torch.load(full_path_to_model_checkpoint)
model.eval()

# Generate, download, plot, and listen to the output

In [ ]:
#@title Generate and download music TXT file with the loaded Model (MIDI-TXT-MIDI)
number_of_tokens_to_generate = 16384 #@param {type:"slider", min:0, max:32768, step:128}
creativity_temperature = 0.8 #@param {type:"slider", min:0.05, max:4, step:0.05}
top_k_prob = 30 #@param {type:"slider", min:0, max:50, step:1}
input_promt = "N" #@param {type:"string"}

%cd /content/

model.to(device)

context = input_promt
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, number_of_tokens_to_generate, temperature=creativity_temperature, sample=True, top_k=top_k_prob)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print('Done! Saving output.txt!')
with open("/content/output.txt", "w") as text_file:
    print(completion, file=text_file)


from google.colab import files
files.download('/content/output.txt')

In [ ]:
#@title Convert to MIDI from TXT (MIDI-TXT-MIDI v.3.5)
number_of_ticks_per_quarter = 425 #@param {type:"slider", min:1, max:1000, step:8}

import MIDI
import tqdm.auto
notes = []
velocities = []
timings = []
durations = []
song_score = []
melody_score = []
with open('/content/output.txt', 'r') as file:
    notestring=file.read()

score = notestring.split(" ")



i=0

z=1

zero_marker = True
if encoding_type == 'score-special-one-byte-encoding': 
  song_score = [number_of_ticks_per_quarter, [['track_name', 0, b'Composed by Artificial Intelligence Model']], [['patch_change', 0, 0, 0], ['patch_change', 0, 3, 40]]]
if encoding_type == 'score-karaoke-one-byte-encoding': 
  song_score = [number_of_ticks_per_quarter, [['track_name', 0, b'Composed by Artificial Intelligence Model']], [['patch_change', 0, 0, 40], ['patch_change', 0, 1, 40]]]

for i in tqdm.auto.tqdm(range(len(score))): 


            note_string_len = len(score[i])
            for j in range(1,200):
                if i+j==len(score):
                    break

            if encoding_type == 'score-special-one-byte-encoding':
              if score[i] == 'N':
                try:
                 
                  song_score[-1].append(['note', 
                                        int(score[i+1][1:]), #Start Time
                                        int(score[i+2][1:]), #Duration
                                        int(score[i+3][1:]), #Channel
                                        int(score[i+4][1:]), #Note
                                        int(score[i+5][1:])]) #Velocity
                            
                except:
                  print("Unknown event: " + score[i] + ' ' + score[i+1])

            if encoding_type == 'score-karaoke-one-byte-encoding':
              if score[i] == 'N':
                try:
                 
                  song_score[-1].append(['note', 
                                        int(score[i+1][1:]), #Start Time
                                        int(score[i+2][1:]), #Duration
                                        int(score[i+3][1:]), #Channel
                                        int(score[i+4][1:]), #Note
                                        int(score[i+5][1:])]) #Velocity                        
                except:
                  print("Unknown event: " + score[i] + ' ' + score[i+1])

              if score[i] == 'L':
                try:
                  song_score[-1].append(['lyric', 
                                        int(score[i+1][1:]), 
                                        score[i+2][1:]])
                  zero_marker = True
                except:
                  print("Unknown event: " + score[i])



if encoding_type == 'score-one-byte-encoding':
  midi_data = MIDI.score2midi(song_score)
  if debug:
    print('Encoding Type: ', encoding_type)
if encoding_type == 'score-special-one-byte-encoding':
   '''midi_data1 = MIDI.score2opus(song_score)
   midi_data2 = MIDI.to_millisecs(midi_data1)
   midi_data = MIDI.opus2midi(midi_data2)'''
   midi_data = MIDI.score2midi(song_score)
if encoding_type == 'score-karaoke-one-byte-encoding':

   midi_data = MIDI.score2midi(song_score)
#else:
#  midi_data = MIDI.opus2midi(song_score)
#  if debug:
#    print('Encoding Type: ', encoding_type)



with open('output.mid', 'wb') as midi_file:
    midi_file.write(midi_data)
    midi_file.close()
print('Done!')

from google.colab import files
files.download('/content/output.mid')

MIDI.score2stats(song_score)

In [ ]:
#@title Analysis of the IO MIDI Datasets
MIDI_DIR = "/content/output.mid"
### https://github.com/brennan2602/FYP

#This file reads in the midi files in a directory, converts them to a string representation
#when in a string representation it then gathers some statistics about the structure of the song
import glob
def get_piano_roll(midifile):
	midi_pretty_format = pretty_midi.PrettyMIDI(midifile)
	piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
	piano_roll = piano_midi.get_piano_roll(fs=20)
	return piano_roll

#uses split encoding scheme (here only encoding the note values)
#works by looping through time increments of the piano roll array and writing the notes being played
#at a given time sample as a number on the corresponding line of a string # is written when no notes played for that
#sample
def encode(arr):
    timeinc=0
    outString=""
    for time in arr:
        notesinc = -1
        #print(time)
        if np.all(time==0):
            outString=outString+"#"
        for vel in arr[timeinc]:
            notesinc=notesinc+1
            if vel != 0:
                noteRep=str(notesinc) + " "
                #print(noteRep)
                outString=outString+noteRep
        outString=outString+"\n"
        timeinc = timeinc+1
    return outString


def getSilences(test):
    test=test[:-1] #removing last line in string (always blank)
    output=test.split("\n") #splitting into array
    res = len(output)
    #initialising counters
    maxcounter=0
    counter=0
    silenceCount=0

    for x in output:
        if x == "#": #when a "#" is seen nothing is being played that sample
            counter=counter+1 #this tracks a streak of silences
            silenceCount+=1 #this tracks total silences
        if x != "#":
            counter=0 #reseting streak
        if counter>maxcounter:
            maxcounter=counter #updating longest silence streak when appropriate
    return maxcounter,silenceCount


#by looking at the length of song and the amount of silences this returns % silence
def getPercentSilence(gen,silences):
    test = gen
    test = test[:-1]
    output = test.split("\n")
    res = len(output)
    percent=silences/res
    return percent


def getStatsNotes(test):
    test=test[:-1] #get rid of blank line at the end
    notes=[]
    output = test.split("\n") #split string on new lines

    #initial values updated while looping through
    maxPerSamp=0
    silenceSamp=0
    notesPlayed=0
    maxNotes=0
    maxVal=0
    minVal=127

    for x in output:
        samp=x.split(" ")
        samp=samp[:-1] #theres a blank result at the end of array from split this indexing removes it
        while "0" in samp:
            samp.remove("0") #sometimes 0 samples exist this removes them as they aren't notes played
        if len(samp)==0:
            silenceSamp+=1 #counting silences
        notesPlayed=notesPlayed+len(samp) #counting notes played
        if len(samp)>0:
            #getting max and min note values at this time step
            minimum=min(samp)
            maximum=max(samp)
            #updating max and min values note values for song if appropriate
            if int(minimum)<minVal:
                minVal=int(minimum)
            if int(maximum)>maxVal:
                maxVal=int(maximum)
        #updating maximum number of notes per sample if appropriate
        if len(samp)>maxNotes:
            maxNotes=len(samp)
    rangeNotes=maxVal-minVal #spread of notes
    avgNotes = notesPlayed / len(output) #average notes per sample
    adjNotes=notesPlayed /(len(output)-silenceSamp) #average notes per sample adjusted to remove silent samples
    return rangeNotes, maxVal, minVal,maxNotes,avgNotes,adjNotes


files=glob.glob(MIDI_DIR)#point towards directory with midi files (here same folder)
print(files)

for f in files:
    print(f)
    pr = get_piano_roll(f) #gets piano roll representation of the midi file
    arr = pr.T
    outString= encode(arr) #gets a string representation of the midi file
    maxsilences, silences = getSilences(outString) #by passing in the encoded string get longest silence and the total
                                                   #number of samples which are silent
    noteRange, maxVal, minVal, maxNotes, avgNotes, adjAvg =getStatsNotes(outString) # getting some stats by looping
                                                                                    # through encoded data
    percentSilence= getPercentSilence(outString,silences) # get % silence from silence / outString length

    #printing out to the user
    print("longest silence is ",maxsilences,"samples long")
    print("silence covers:",round(percentSilence,4),"%")
    print("notes span range:",noteRange)
    print("max note value:",maxVal)
    print("min note value:",minVal)
    print("average number of notes per sample:",round(avgNotes,4))
    print("average number of notes per sample (adjusted to remove silence samples):",round(adjAvg,4))
    print("max number of notes played in a sample:",maxNotes)
    print("\n")

#NOTE some minor discrepencies vs reading in from generated file directly
#However this does provide a uniform check to use for songs generated by both encoding schemes
#Can also be used to evaluate training file
#uses split encoding to get the text representation for ease of development

In [ ]:
#@title Plot, Graph, and Listen to the Output :)
graphs_length_inches = 18 #@param {type:"slider", min:0, max:20, step:1}
notes_graph_height = 6 #@param {type:"slider", min:0, max:20, step:1}
highest_displayed_pitch = 92 #@param {type:"slider", min:1, max:128, step:1}
lowest_displayed_pitch = 24 #@param {type:"slider", min:1, max:128, step:1}

midi_data = pretty_midi.PrettyMIDI('/content/output.mid')

def plot_piano_roll(pm, start_pitch, end_pitch, fs=100):
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))



roll = np.zeros([int(graphs_length_inches), 128])
# Plot the output

track = Multitrack('/content/output.mid', name='track')
plt.figure(figsize=[graphs_length_inches, notes_graph_height])
fig, ax = track.plot()
fig.set_size_inches(graphs_length_inches, notes_graph_height)
plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, int(lowest_displayed_pitch), int(highest_displayed_pitch))
plt.show(block=False)


FluidSynth("/content/font.sf2", 16000).midi_to_audio('/content/output.mid', '/content/output.wav')
Audio('/content/output.wav', rate=16000)

## Congrats! :) You did it :)

In [ ]:
#@title Make a nice Arc diagram of the output to show friends and family :)
multi_track_input = True #@param {type:"boolean"}
%cd /content/arc-diagrams/

midi_file = '/content/output.mid'
plot_title = "Amazing GPT2 Piano Output Arc Diagram"

# midi_file = 'midis/fuer_elise.mid'
# plot_title = "Für Elise (Beethoven)"

def stringify_notes(midi_file, track_number):

    mid = MidiFile(midi_file)
    track_notes = {}
    for i, track in enumerate(mid.tracks):
        track_notes[i] = ''
        for msg in track:
            if( msg.type == 'note_on'):
                track_notes[i] += str(msg.note) +'n'
            if( msg.type == 'note_off'):
                track_notes[i] += str(msg.note) +'f'
    return track_notes[track_number]

if multi_track_input:
  try:
    plot_arc_diagram(stringify_notes(midi_file, 1), plot_title)
    if debug: 
      print('Debug mode')
    print('Track 1 Arc Diagram')
  except:
    plot_arc_diagram(stringify_notes(midi_file, 0), plot_title)
    if debug: 
      print('Debug mode')
    print('Track 0 Arc Diagram')

Image('output.png')

if multi_track_input:
  try:
   plot_arc_diagram(stringify_notes(midi_file, 2), plot_title)
   if debug: print('Debug mode')
   print('Track 2 Arc Digram')
   Image('output.png')
   plot_arc_diagram(stringify_notes(midi_file, 3), plot_title)
   if debug: print('Debug mode')
   print('Track 3 Arc Diagram')
   Image('output.png')
  except:
    print('Error in processing multiple tracks. Sorry.')
files.download('/content/arc-diagrams/output.png')

# Save the model and/or output to Google Drive (Standard GD connect code)

In [ ]:
#@title Mount Google Drive
drive.mount('/content/drive')